# Load Demo with DAG

In this we demonstrate loading of a sample data file [reduced_sample_data.json](../../data/reduced_sample_data.json) and showcase all the 
various resources that gets populated. We now are demonstrating using DAG

#### Pre-requisite
It is assumed that the setup steps are completed successfully. These are:
 - creating the database, schemas, stages
 - defining the external stage
 - defining the functions and stored procedures
 - defining the external tables and views

If you had not done this, use the streamlit (./bin/run_app.sh) to create these as defined in the subpage 'Setup'

In [ ]:
# Initialization block
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

In [88]:
import pandas as pd
import os
from datetime import datetime
from datetime import timedelta
import time
import re

pd.set_option('display.max_colwidth', None)

def get_basename_of_datafile(p_datafile:str) -> str:
    base = os.path.basename(p_datafile)
    fl_base = os.path.splitext(base)
    return fl_base[0]

def get_cleansed_file_basename(p_datafile):
    fl_basename = get_basename_of_datafile(p_datafile)
    # Replace all non alphanumeric characters with _
    fl_name = re.sub('[^0-9a-zA-Z]+', '_', fl_basename)
    return fl_name

# INPUT_DATA_STAGE = config['APP_DB']['ext_stage']
INPUT_DATA_STAGE = 'data_stg'
DATA_STAGE_FOLDER = config['APP_DB']['folder_data']

DATA_FILE = '2022_10_01_priority_health_HMO_in-network-rates.zip'
# DATA_FILE = 'reduced_sample_data.json'
DATA_FILE_BASENAME = get_basename_of_datafile(DATA_FILE)
DATA_FILE_BASENAME_CLEANSED = get_cleansed_file_basename(DATA_FILE)

TARGET_DATA_STAGE = config['APP_DB']['ext_stage']
TARGET_FOLDER = config['APP_DB']['folder_parsed']

In [80]:
display(Markdown("### Cleanup block"))
# We will cleanup specific resources and artifacts from possible previous runs.

stmts = [
    f''' delete from segment_task_execution_status where data_file = '{DATA_FILE}'; '''
    ,f''' delete from task_to_segmentids where data_file = '{DATA_FILE}'; '''
    ,f''' delete from in_network_rates_file_header where data_file = '{DATA_FILE}'; '''
    ,f''' delete from in_network_rates_segment_header where data_file = '{DATA_FILE}'; '''
    ,f''' alter stage {INPUT_DATA_STAGE} refresh; '''
]    
    
print(' truncating tables ...')
for stmt in stmts:
    sp_session.sql(stmt).collect()

print(f''' cleaning up files in external stage under path {TARGET_FOLDER}/{DATA_FILE_BASENAME}/ ...''')

stmt = f''' select relative_path from directory(@{TARGET_DATA_STAGE}) where relative_path like '%{DATA_STAGE_FOLDER}/{DATA_FILE_BASENAME}/%'; '''
files = sp_session.sql(stmt).collect()
for r in files:
    stmt = f''' remove @{TARGET_DATA_STAGE}/{r['RELATIVE_PATH']}; '''
    sp_session.sql(stmt).collect()

### Cleanup block

 truncating tables ...
 cleaning up files in external stage under path data/2022_10_01_priority_health_HMO_in-network-rates/ ...


---
## Data loading
We will be loading the segments and file header using DAG. 

In [86]:
# we build out the DAG
df = sp_session.call('in_network_rates_dagbuilder' ,f'{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}' ,DATA_FILE 
    ,f"@{TARGET_DATA_STAGE}/{TARGET_FOLDER}" , 200 ,config['SNOW_CONN']['warehouse'])

sp_session.sql(f''' alter stage {TARGET_DATA_STAGE} refresh; ''').collect()
print(' Status of execution')
print(df)

 Status of execution
{
  "data_file": "2022_10_01_priority_health_HMO_in-network-rates.zip",
  "root_task": "DAG_ROOT_2022_10_01_priority_health_HMO_in_network_rates",
  "status": true,
  "task_matrix_shape": [
    5,
    15
  ],
  "term_task": "TERM_tsk_2022_10_01_priority_health_HMO_in_network_rates"
}


The above operation results in defining the DAG in Snowflake like here. The task names are specific to the data file being parsed.
![](../../doc/images/task_dags.png)

In [89]:
# Next we invoke the DAG

start_time = time.time()
print(f'Started at: {datetime.now().strftime("%H:%M:%S")}')

sql_stmts = [
    f''' alter warehouse {config['SNOW_CONN']['warehouse']} set warehouse_size = MEDIUM; '''
    # ,f''' truncate table segment_task_execution_status; '''
    # ,f''' truncate table task_to_segmentids; '''
    # ,f''' truncate table in_network_rates_file_header; '''
    # ,f''' truncate table in_network_rates_segment_header; '''
    ,f''' execute task DAG_ROOT_{DATA_FILE_BASENAME_CLEANSED}; '''
]
for stmt in sql_stmts:
    print(stmt)
    sp_session.sql(stmt).collect()

end_time = time.time()
print(f'Ended at: {datetime.now().strftime("%H:%M:%S")}')

elapsed_time = end_time - start_time
elapsed = str(timedelta(seconds=elapsed_time))
print(f'Elapsed: {elapsed}')

Started at: 07:46:57
 alter warehouse DEMO_BUILD_WH set warehouse_size = MEDIUM; 
 execute task DAG_ROOT_2022_10_01_priority_health_HMO_in_network_rates; 
Ended at: 07:46:58
Elapsed: 0:00:01.117523


---
## Inspection (TODO)

In [101]:
import time

continue_sleeping = True

while continue_sleeping == True: 
    time.sleep(2*60)

    sql_stmt = f'''
        select *
        from current_segment_parsing_tasks_v
        where l.data_file = '{DATA_FILE}'
        ;
    '''
    df_running = sp_session.table('current_segment_parsing_tasks_v').to_pandas()
    
    len_running_count = len(df_running)
    print(f'Current running tasks [{len_running_count}]...')
    display(df_running)
    
    sql_stmt = f'''
        select 
            count(l.*) as l_count
            ,count(r.*) as r_count
            ,l_count - r_count as row_count_diff
            -- l.*
        from segment_task_execution_status as l
            full outer join segments_counts_for_datafile_v as r
                on r.task_name = l.task_name
        where not (l.task_name  like any ('DAG_%' ,'TERM_%' ,'%T_FH_%' ))
            and JSON_EXTRACT_PATH_TEXT(l.task_ret_status ,'task_ignored_parsing') is null
            and l.data_file = '{DATA_FILE}'
    '''
    df = sp_session.sql(sql_stmt).to_pandas()
    row_count = df['ROW_COUNT_DIFF'][0]
    finished_row_count = df['R_COUNT'][0]
    print(f'Finished tasks [{row_count}] ...')
    if row_count == 0:
        print('Finished sleeping ...')
        continue_sleeping = False
        break

    display(df)
    print('\n ------------------------------ ')

Current running tasks [5]...


,DATA_FILE,TASK_NAME,START_TIME,END_TIME,TASK_RET_STATUS,INSERTED_AT,BUCKET,ASSIGNED_TASK_NAME,FROM_IDX,TO_IDX,SEGMENTS_RECORD_COUNT
0,2022_10_01_priority_health_HMO_in-network-rates.zip,INDSOL_CMSGOV_PRICING.PUBLIC.T_2022_10_01_PRIORITY_HEALTH_HMO_IN_NETWORK_RATES_201_400,2023-01-06 04:47:19.332,NaT,None,2023-01-06 04:47:19.332,1,T_2022_10_01_priority_health_HMO_in_network_rates_201_400,201,400,199
1,2022_10_01_priority_health_HMO_in-network-rates.zip,INDSOL_CMSGOV_PRICING.PUBLIC.T_2022_10_01_PRIORITY_HEALTH_HMO_IN_NETWORK_RATES_601_800,2023-01-06 04:47:10.698,NaT,None,2023-01-06 04:47:10.698,3,T_2022_10_01_priority_health_HMO_in_network_rates_601_800,601,800,199
2,2022_10_01_priority_health_HMO_in-network-rates.zip,INDSOL_CMSGOV_PRICING.PUBLIC.T_2022_10_01_PRIORITY_HEALTH_HMO_IN_NETWORK_RATES_801_1000,2023-01-06 04:47:10.680,NaT,None,2023-01-06 04:47:10.680,4,T_2022_10_01_priority_health_HMO_in_network_rates_801_1000,801,1000,199
3,2022_10_01_priority_health_HMO_in-network-rates.zip,INDSOL_CMSGOV_PRICING.PUBLIC.T_2022_10_01_PRIORITY_HEALTH_HMO_IN_NETWORK_RATES_401_600,2023-01-06 04:47:19.306,NaT,None,2023-01-06 04:47:19.306,2,T_2022_10_01_priority_health_HMO_in_network_rates_401_600,401,600,199
4,2022_10_01_priority_health_HMO_in-network-rates.zip,INDSOL_CMSGOV_PRICING.PUBLIC.T_2022_10_01_PRIORITY_HEALTH_HMO_IN_NETWORK_RATES_1001_1200,2023-01-06 05:00:18.927,NaT,None,2023-01-06 05:00:18.927,5,T_2022_10_01_priority_health_HMO_in_network_rates_1001_1200,1001,1200,199


Finished tasks [0] ...
Finished sleeping ...


In [91]:
display(Markdown("The table in_network_rates_file_header holds the file header elements"))

df = (sp_session.table('in_network_rates_file_header')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .to_pandas())

display(df)

The table in_network_rates_file_header holds the file header elements

,DATA_FILE,HEADER,INSERTED_AT
0,reduced_sample_data.json,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""last_updated_on"": ""2022-10-01"",\n ""reporting_entity_name"": ""Priority Health"",\n ""reporting_entity_type"": ""health insurance issuer"",\n ""total_segments"": 10,\n ""version"": ""http://json-schema.org/draft-07/schema#""\n}",2023-01-06 04:09:24.412


In [92]:
print(' The HEADER has the following data ')
json.loads(df['HEADER'][0])

 The HEADER has the following data 


{'DATA_FILE': 'reduced_sample_data.json',
 'last_updated_on': '2022-10-01',
 'reporting_entity_name': 'Priority Health',
 'reporting_entity_type': 'health insurance issuer',
 'total_segments': 10,
 'version': 'http://json-schema.org/draft-07/schema#'}

In [93]:
display(Markdown("The table in_network_rates_segment_header lists the various segments that were loaded"))

df = (sp_session.table('in_network_rates_segment_header')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

The table in_network_rates_segment_header lists the various segments that were loaded

,DATA_FILE,SEGMENT_ID,NEGOTIATED_RATES_INFO,NEGOTIATED_RATES_COUNT,BUNDLED_CODES_COUNT,COVERED_SERVICES_COUNT,INSERTED_AT
0,2022_10_01_priority_health_HMO_in-network-rates.zip,ffs::ms-drg::029::,"{\n ""DATA_FILE"": ""2022_10_01_priority_health_HMO_in-network-rates.zip"",\n ""SEGMENT_ID"": ""ffs::ms-drg::029::"",\n ""SEQ_NO"": 201,\n ""billing_code"": ""029"",\n ""billing_code_type"": ""MS-DRG"",\n ""billing_code_type_version"": """",\n ""description"": ""TRAUM STUPOR&COMA,COMA<1HR,AGE>17 W/O CC "",\n ""name"": ""TRAUM STUPOR&COMA,COMA<1HR,AGE>17 W/O CC"",\n ""negotiation_arrangement"": ""ffs""\n}",3,-1,-1,2023-01-06 04:48:23.483
1,reduced_sample_data.json,ffs::cpt::j9144::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::j9144::"",\n ""SEQ_NO"": 10,\n ""billing_code"": ""J9144"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Injection, daratumumab, 10 mg and hyaluronidase-fihj"",\n ""name"": ""Injection, daratumumab, 10 mg and hyaluronidase-fihj"",\n ""negotiation_arrangement"": ""ffs""\n}",573,-1,-1,2023-01-06 04:09:57.396
2,2022_10_01_priority_health_HMO_in-network-rates.zip,ffs::cpt::67042::,"{\n ""DATA_FILE"": ""2022_10_01_priority_health_HMO_in-network-rates.zip"",\n ""SEGMENT_ID"": ""ffs::cpt::67042::"",\n ""SEQ_NO"": 18,\n ""billing_code"": ""67042"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Vitrectomy, Mechanical, Pars Plana Approach; with Removal of Internal Limiting Membrane of Retina"",\n ""name"": ""Vitrectomy, Mechanical, Pars Plana Approach; with Removal of Internal Limiting Membrane of Retina"",\n ""negotiation_arrangement"": ""ffs""\n}",11954,-1,-1,2023-01-06 04:48:44.685
3,reduced_sample_data.json,ffs::cpt::15004::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::15004::"",\n ""SEQ_NO"": 7,\n ""billing_code"": ""15004"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""PREP SITE F/S/N/H/F/G/M/D GT 1ST 100 SQ CM/1PCT"",\n ""name"": ""PREP SITE F/S/N/H/F/G/M/D GT 1ST 100 SQ CM/1PCT"",\n ""negotiation_arrangement"": ""ffs""\n}",9227,-1,-1,2023-01-06 04:09:47.147
4,reduced_sample_data.json,ffs::cpt::76391::,"{\n ""DATA_FILE"": ""reduced_sample_data.json"",\n ""SEGMENT_ID"": ""ffs::cpt::76391::"",\n ""SEQ_NO"": 5,\n ""billing_code"": ""76391"",\n ""billing_code_type"": ""CPT"",\n ""billing_code_type_version"": """",\n ""description"": ""Magnetic resonance (eg, vibration) elastography"",\n ""name"": ""Magnetic resonance (eg, vibration) elastography"",\n ""negotiation_arrangement"": ""ffs""\n}",1784,-1,-1,2023-01-06 04:09:40.527


In [94]:
print(' The NEGOTIATED_RATES_INFO has the following data ')
json.loads(df['NEGOTIATED_RATES_INFO'][0])

 The NEGOTIATED_RATES_INFO has the following data 


{'DATA_FILE': '2022_10_01_priority_health_HMO_in-network-rates.zip',
 'SEGMENT_ID': 'ffs::ms-drg::029::',
 'SEQ_NO': 201,
 'billing_code': '029',
 'billing_code_type': 'MS-DRG',
 'billing_code_type_version': '',
 'description': 'TRAUM STUPOR&COMA,COMA<1HR,AGE>17 W/O CC   ',
 'name': 'TRAUM STUPOR&COMA,COMA<1HR,AGE>17 W/O CC',
 'negotiation_arrangement': 'ffs'}

In [95]:
para = f''' 
The parsed data are stored as parquet files in the external stage at: @{TARGET_DATA_STAGE}/{TARGET_FOLDER}. The directory structure follows the format:

<ext stage>/<folder_parsed>/<data file basename (ex: reduced_sample_data)/<segment_id>/<segment type (negotiated_rates)>/data_<seq_no>_<chunk_no>.parquet.gz

Description:
- ext stage : external stage name
- folder_parsed : configured value of APP_DB.folder_parsed in config.ini
- data file basename : the data file basename, without the file extension
- segment_id : a unique identifier for the segment, this is a composite key of <negotiation_arrangement>::<billing_code_type>::<billing_code>::<billing_code_type_version>
- segment type : indicates the segment children type, this could be either of negotiated_rates/bundled_codes/covered_services
- The file which will contain the record, stored in parquet file.
'''
print(para)

stmt = f''' select relative_path from directory(@{TARGET_DATA_STAGE}) where relative_path like '%{TARGET_FOLDER}/{DATA_FILE_BASENAME}%' limit 5; '''
df = sp_session.sql(stmt).to_pandas()
display(df)

 
The parsed data are stored as parquet files in the external stage at: @ext_data_stg/raw_parsed. The directory structure follows the format:

<ext stage>/<folder_parsed>/<data file basename (ex: reduced_sample_data)/<segment_id>/<segment type (negotiated_rates)>/data_<seq_no>_<chunk_no>.parquet.gz

Description:
- ext stage : external stage name
- folder_parsed : configured value of APP_DB.folder_parsed in config.ini
- data file basename : the data file basename, without the file extension
- segment_id : a unique identifier for the segment, this is a composite key of <negotiation_arrangement>::<billing_code_type>::<billing_code>::<billing_code_type_version>
- segment type : indicates the segment children type, this could be either of negotiated_rates/bundled_codes/covered_services
- The file which will contain the record, stored in parquet file.



,RELATIVE_PATH
0,raw_parsed/reduced_sample_data/ffs::cpt::15004::/negotiated_rates/data_7_0.parquet.gz
1,raw_parsed/reduced_sample_data/ffs::cpt::15004::/negotiated_rates/data_7_1.parquet.gz
2,raw_parsed/reduced_sample_data/ffs::cpt::33235::/negotiated_rates/data_8_0.parquet.gz
3,raw_parsed/reduced_sample_data/ffs::cpt::33235::/negotiated_rates/data_8_1.parquet.gz
4,raw_parsed/reduced_sample_data/ffs::cpt::62322::/negotiated_rates/data_1_0.parquet.gz


In [96]:
para = f''' 
The parquet file can be queried inside Snowflake via external table: ext_negotiated_arrangments_staged. Each of the folder structure is partitioned to seperate columns , which can help with pruning to specific segments.
The value contains the negotiated_rates and other children elements stored in JSON format
'''
print(para)

pd.set_option('display.max_colwidth', 25)
df = (sp_session.table('ext_negotiated_arrangments_staged')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)


 
The parquet file can be queried inside Snowflake via external table: ext_negotiated_arrangments_staged. Each of the folder structure is partitioned to seperate columns , which can help with pruning to specific segments.
The value contains the negotiated_rates and other children elements stored in JSON format



,VALUE,P_DATA_FL,P_SEGMENT_ID,P_NEGOTIATION_ARRANGEMENT,P_BILLING_CODE_TYPE,P_BILLING_CODE,P_BILLING_CODE_TYPE_VERSION,P_SEGMENT_TYPE
0,"{\n ""CHUNK_NO"": 1,\n...",reduced_sample_data,ffs::cpt::33235::,ffs,cpt,33235,,negotiated_rates
1,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::a4614::,ffs,cpt,a4614,,negotiated_rates
2,"{\n ""CHUNK_NO"": 1,\n...",reduced_sample_data,ffs::cpt::92583::,ffs,cpt,92583,,negotiated_rates
3,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::92583::,ffs,cpt,92583,,negotiated_rates
4,"{\n ""CHUNK_NO"": 0,\n...",reduced_sample_data,ffs::cpt::33235::,ffs,cpt,33235,,negotiated_rates


In [97]:
print(' a sample view of one of the records')
j = json.loads(df['VALUE'][0])
print(j)

 a sample view of one of the records
{'CHUNK_NO': 1, 'DATA_FILE': 'reduced_sample_data.json', 'NEGOTIATED_RATES': [{'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 975.15, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}, {'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 3259.44, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}], 'provider_groups': [{'npi': [1609821081], 'tin': {'type': 'ein', 'value': '382800065'}}]}, {'negotiated_prices': [{'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 975.15, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}, {'billing_class': 'professional', 'expiration_date': '9999-12-31', 'negotiated_rate': 3259.44, 'negotiated_type': 'fee schedule', 'service_code': ['11', '22']}], 'provider_groups': [{'npi': [1427430842], 'tin': {'type': 'ein', 'value': '382800065'}}]}, {'negotiated_

In [98]:
display(Markdown("The table segment_task_execution_status contains the audit of execution for the above stored procedures"))

pd.set_option('display.max_colwidth', None)
df = (sp_session.table('segment_task_execution_status')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

The table segment_task_execution_status contains the audit of execution for the above stored procedures

,DATA_FILE,TASK_NAME,START_TIME,END_TIME,TASK_RET_STATUS,INSERTED_AT
0,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_201_400,2023-01-06 04:09:22.706,2023-01-06 04:09:26.582,"{""data_file"": ""reduced_sample_data.json"", ""start_rec_num"": 201, ""end_rec_num"": 400, ""Parsing_error"": """", ""last_seg_no"": 10, ""EOF_Reached"": True, ""task_ignored_parsing"": False, ""elapsed"": ""=> 0:00:02.662760 ""}",2023-01-06 04:09:22.706
1,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_2201_2400,2023-01-06 04:09:46.363,2023-01-06 04:09:47.241,"{""data_file"": ""reduced_sample_data.json"", ""start_rec_num"": 2201, ""end_rec_num"": 2400, ""task_ignored_parsing"": True, ""task_parsing_ignore_message"": ""The segment range is greater than the total segments -2 in the file, hence ignore further parsing"", ""elapsed"": ""=> 0:00:00.372729 ""}",2023-01-06 04:09:46.363
2,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_13601_13800,2023-01-06 04:12:10.514,2023-01-06 04:12:12.550,"{""data_file"": ""reduced_sample_data.json"", ""start_rec_num"": 13601, ""end_rec_num"": 13800, ""task_ignored_parsing"": True, ""task_parsing_ignore_message"": ""The segment range is greater than the total segments -2 in the file, hence ignore further parsing"", ""elapsed"": ""=> 0:00:00.404196 ""}",2023-01-06 04:12:10.514
3,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_9601_9800,2023-01-06 04:11:22.449,2023-01-06 04:11:23.598,"{""data_file"": ""reduced_sample_data.json"", ""start_rec_num"": 9601, ""end_rec_num"": 9800, ""task_ignored_parsing"": True, ""task_parsing_ignore_message"": ""The segment range is greater than the total segments -2 in the file, hence ignore further parsing"", ""elapsed"": ""=> 0:00:00.504701 ""}",2023-01-06 04:11:22.449
4,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_8001_8200,2023-01-06 04:11:34.416,2023-01-06 04:11:36.343,"{""data_file"": ""reduced_sample_data.json"", ""start_rec_num"": 8001, ""end_rec_num"": 8200, ""task_ignored_parsing"": True, ""task_parsing_ignore_message"": ""The segment range is greater than the total segments -2 in the file, hence ignore further parsing"", ""elapsed"": ""=> 0:00:00.337956 ""}",2023-01-06 04:11:34.416


In [99]:
display(Markdown("The view segments_counts_for_datafile_v, based of table segment_task_execution_status, will help to identify the number of negotiated_arrangement segments for a specific data file. This view is populated once all the segments in a specific data files are parsed out"))

df = (sp_session.table('segments_counts_for_datafile_v')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

The view segments_counts_for_datafile_v, based of table segment_task_execution_status, will help to identify the number of negotiated_arrangement segments for a specific data file. This view is populated once all the segments in a specific data files are parsed out

,DATA_FILE,TASK_NAME,START_REC_NUM,END_REC_NUM,TOTAL_NO_OF_SEGMENTS,ELAPSED,PARSING_ERROR,EOF_REACHED
0,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_0_200,0,200,10,0,,true
1,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_201_400,201,400,10,0,,true
2,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_601_800,601,800,10,0,,true
3,reduced_sample_data.json,INDSOL_CMSGOV_PRICING.PUBLIC.T_REDUCED_SAMPLE_DATA_801_1000,801,1000,10,0,,true


--- 
### Closeout

    With that we are finished this section of the demo setup

In [100]:
# sp_session.close()
print('Finished!!!')

Finished!!!
